In [ ]:
"""
PPE Detection Model Training on Google Colab
Fast training with free GPU!
"""

# ============================================================
# STEP 1: Setup Environment
# ============================================================

print("="*60)
print("   STEP 1: Installing Requirements")
print("="*60 + "\n")

!pip install roboflow ultralytics -q

print("✅ Requirements installed!\n")

# ============================================================
# STEP 2: Download Dataset from Roboflow
# ============================================================

print("="*60)
print("   STEP 2: Downloading Dataset")
print("="*60 + "\n")

from roboflow import Roboflow

# ENTER YOUR API KEY HERE
ROBOFLOW_API_KEY = ""  # ← Paste your API key

if ROBOFLOW_API_KEY == "YOUR_API_KEY":
    print("❌ ERROR: Please set your Roboflow API key above!")
    print("   Get it from: https://app.roboflow.com → Settings → API\n")
else:
    print("📦 Connecting to Roboflow...")
    rf = Roboflow(api_key=ROBOFLOW_API_KEY)

    print("📦 Downloading PPE dataset...")
    project = rf.workspace("roboflow-universe-projects").project("construction-site-safety")
    dataset = project.version(27).download("yolov8")

    print(f"✅ Dataset downloaded to: {dataset.location}\n")

    DATASET_PATH = dataset.location
    DATA_YAML = f"{DATASET_PATH}/data.yaml"

# ============================================================
# STEP 3: Check Dataset
# ============================================================

print("="*60)
print("   STEP 3: Dataset Information")
print("="*60 + "\n")

import os
import yaml

if os.path.exists(DATA_YAML):
    with open(DATA_YAML, 'r') as f:
        data_config = yaml.safe_load(f)

    print(f"📊 Classes: {data_config.get('names', [])}")
    print(f"📁 Training images: {data_config.get('train', 'N/A')}")
    print(f"📁 Validation images: {data_config.get('val', 'N/A')}")
    print()
else:
    print("❌ Dataset YAML not found!")

# ============================================================
# STEP 4: Train Model
# ============================================================

print("="*60)
print("   STEP 4: Training Model")
print("="*60 + "\n")

from ultralytics import YOLO
import torch

# Check GPU availability
if torch.cuda.is_available():
    print(f"✅ GPU Available: {torch.cuda.get_device_name(0)}")
    device = 0  # Use GPU
else:
    print("⚠️  No GPU detected, using CPU (will be slower)")
    device = 'cpu'

print()

# Load base model
print("🔄 Loading YOLOv8 base model...")
model = YOLO('yolov8n.pt')  # Nano model (fastest)

print("🚀 Starting training...")
print("⏱️  This will take 2-5 minutes with GPU\n")

# Train!
results = model.train(
    data=DATA_YAML,
    epochs=50,           # Number of training cycles
    imgsz=640,           # Image size
    batch=16,            # Batch size (adjust if GPU runs out of memory)
    device=device,       # GPU or CPU
    project='ppe_training',
    name='model',
    patience=10,         # Early stopping
    save=True,
    plots=True,
    verbose=True
)

print("\n✅ Training completed!\n")

# ============================================================
# STEP 5: Test Model
# ============================================================

print("="*60)
print("   STEP 5: Testing Model")
print("="*60 + "\n")

# Load trained model
trained_model_path = 'ppe_training/model/weights/best.pt'

if os.path.exists(trained_model_path):
    print(f"✅ Trained model found: {trained_model_path}")

    # Test on a sample image
    test_model = YOLO(trained_model_path)

    # Find a test image
    test_images = []
    if 'valid' in data_config:
        valid_dir = f"{DATASET_PATH}/valid/images"
        if os.path.exists(valid_dir):
            test_images = [f"{valid_dir}/{img}" for img in os.listdir(valid_dir) if img.endswith(('.jpg', '.png'))][:5]

    if test_images:
        print(f"\n🔍 Testing on {len(test_images)} sample images...\n")

        for img_path in test_images:
            results = test_model(img_path)
            results[0].show()  # Display result
            print(f"   ✓ Tested: {os.path.basename(img_path)}")
    else:
        print("⚠️  No test images found")
else:
    print(f"❌ Trained model not found at: {trained_model_path}")

# ============================================================
# STEP 6: Download Model
# ============================================================

print("\n" + "="*60)
print("   STEP 6: Download Your Model")
print("="*60 + "\n")

if os.path.exists(trained_model_path):
    print("✅ Your trained model is ready!")
    print(f"📁 Location: {trained_model_path}")
    print("\n📥 Download it using the Colab file browser:")
    print("   1. Click the folder icon on the left")
    print("   2. Navigate to: ppe_training/model/weights/")
    print("   3. Right-click 'best.pt' → Download")
    print("\n💾 Save it as 'ppe_local.pt' on your Mac")
    print("="*60 + "\n")

    # Show training metrics
    print("📊 TRAINING RESULTS:")
    print("="*60)

    results_csv = 'ppe_training/model/results.csv'
    if os.path.exists(results_csv):
        import pandas as pd
        df = pd.read_csv(results_csv)

        # Show final metrics
        final = df.iloc[-1]
        print(f"\nFinal Epoch: {int(final.get('epoch', 0))}")

        metrics = ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
        for metric in metrics:
            if metric in final:
                print(f"{metric.split('/')[-1]}: {final[metric]:.3f}")

    # Display training charts
    print("\n📈 Training charts saved to: ppe_training/model/")

    # Show confusion matrix if exists
    confusion_matrix_path = 'ppe_training/model/confusion_matrix.png'
    if os.path.exists(confusion_matrix_path):
        from IPython.display import Image, display
        print("\n📊 Confusion Matrix:")
        display(Image(filename=confusion_matrix_path))

    print("\n" + "="*60)
    print("✅ TRAINING COMPLETE!")
    print("="*60)
    print("\n🎯 Next steps:")
    print("   1. Download 'best.pt' from Colab")
    print("   2. Save it as 'ppe_local.pt' on your Mac")
    print("   3. Run your detection app with the local model")
    print("\n💡 Your model is now trained on PPE detection!")
    print("="*60 + "\n")

else:
    print("❌ Training may have failed. Check errors above.\n")

# ============================================================
# BONUS: Quick Validation
# ============================================================

print("="*60)
print("   BONUS: Model Validation")
print("="*60 + "\n")

if os.path.exists(trained_model_path):
    val_model = YOLO(trained_model_path)

    print("🔍 Running validation on test set...")
    metrics = val_model.val()

    print("\n📊 Validation Metrics:")
    print(f"   mAP50: {metrics.box.map50:.3f}")
    print(f"   mAP50-95: {metrics.box.map:.3f}")
    print(f"   Precision: {metrics.box.mp:.3f}")
    print(f"   Recall: {metrics.box.mr:.3f}")
    print()

   STEP 1: Installing Requirements

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 84.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 97.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 113.6 MB/s eta 0:00:00
✅ Requirements installed!

   STEP 2: Downloading Dataset

📦 Connecting to Roboflow...
📦 Downloading PPE dataset...
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Construction-Site-Safety-27 in yolov8:: 100%|██████████| 5610/5610 [00:00<00:00, 6720.95it/s]


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
✅ Dataset downloaded to: /content/Construction-Site-Safety-27

   STEP 3: Dataset Information

📊 Classes: ['Hardhat', 'Mask', 'NO-Hardhat', 'NO-Mask', 'NO-Safety Vest', 'Person', 'Safety Cone', 'Safety Vest', 'machinery', 'vehicle']
📁 Training images: ../train/images
📁 Validation images: ../valid/images

   STEP 4: Training Model

✅ GPU Available: Tesla T4

🔄 Loading YOLOv8 base model...
🚀 Starting training...
⏱️  This will take 2-5 minutes with GPU

WARNING ⚠️ 'name=model' automatically updated to 'name=yolov8n'.
Ultralytics 8.4.9 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=ran

In [ ]:
from google.colab import files as colab_files
import shutil
import os

model_path = None
for root, dirs, files_list in os.walk('/content/runs'):
    if 'best.pt' in files_list:
        model_path = os.path.join(root, 'best.pt')
        break

if model_path:
    print(f"✅ Found: {model_path}")

    # Copy to root for easy download
    shutil.copy(model_path, '/content/ppe_local.pt')
    print(f"✅ Copied to: /content/ppe_local.pt")

    # Get file size
    file_size = os.path.getsize('/content/ppe_local.pt') / (1024 * 1024)
    print(f"📦 Size: {file_size:.1f} MB")

    print("\n📥 Starting download...")
    colab_files.download('/content/ppe_local.pt')

    print("\n✅ Download complete!")
    print("💾 Save it as 'ppe_local.pt' in your Mac project folder")
else:
    print("❌ Model not found")

✅ Found: /content/runs/detect/ppe_training/yolov8n/weights/best.pt
✅ Copied to: /content/ppe_local.pt
📦 Size: 6.0 MB

📥 Starting download...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ Download complete!
💾 Save it as 'ppe_local.pt' in your Mac project folder
